In [ ]:
# default_exp interface.deribit_interface

# module issu de 00_deribit_interface

> module du premier notebook de test

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello('Sylvain')

'Hello Sylvain!'

In [ ]:
assert say_hello('Hedi') == 'Hello Hedi!'